In [1]:
import copy
class Bittensor:
    # NOTE(const): This is a python template for the Bittensor EOS contract. It is written to approximate the
    # Matrix multiplication emission system from above but using serial transactions which distribute the execution 
    # cost across all the network nodes.
    
    def __init__(self): 
        self.edges = {} # id --> edges.
        self.emit_height = {} # id --> last emit block.
        self.block_height = 0 # chain height.
        
        self.stake = {} # id --> stake
        self.next_stake = {} # temporary version of self.stake (only for testing.)
        self.total_stake = 0 # total supply of staked token.
        self.next_total_stake = 0 # temporary version of self.total_supply (only for testing.)
        
        
    def subscribe(self, this_identity):
        # NOTE(const): Subscribing a neuron to the metagraph will involve locking token funds in the main EOS contract
        # and shifting them into the metagraph to act as stake. The token contract is thus two tokens, one which acts 
        # like a normal EOS token, it can be transfered easily, and tokens which act as stake within the metagraph.
        # When a node "unsubscribes" the stake that has been gained through metagraph emission is transfered back to the
        # participants account on EOS.
        
        # TODO(const): It is not yet obvious if tokens that are held out of the metagraph should be attaining emission.
        # If they are not, then it seems like people are likely to hold their tokens in the metagraph without running a 
        # neuron. This will clutter the graph. It would be better if only participants were offered the opportunity to attain
        # mining emissions.
        self.next_total_stake += 1        
        self.next_stake[this_identity] = 1
        
        
        self.edges[this_identity] = [(this_identity, 1.0)]
        self.emit_height[this_identity] = self.block_height
        
    def inc_height(self):
        # Note(const): This function will not be included in the EOS contract. This is used to keep all the nodes in lockstep,
        # a.k.a at the same height and using the same denominator in ( self.stake[id] / self.total_stake).
        # In reality these will differ and it will give nodes an advantage if they are able to call the emit function
        # before the total_stake term is incremented by their competitors. This should give the network incentive 
        # to continually emit tokens (remember that this is costly for the calling node.)
        self.stake = copy.deepcopy(self.next_stake)
        self.total_stake = self.next_total_stake
        
        self.block_height += 1
        
    def emit(self, this_identity, edges):
        # NOTE(const): We are wrapping the emission function with the update attribution function. 
        # This can be done because we require an emission call BEFORE a node changes his edge set in the graph.
        # hence they are essentially the same call.
                
        # Edge set checks.
        assert(edges[0][0] == this_identity)
        assert(abs(sum([w for i, w in edges]) - 1.0) < 0.001 )

        # Get emission and apply it.
        this_emission = self._get_emission(this_identity)
        self._do_emit(this_identity, this_emission)

        # Update to the new edge set.
        self.edges[this_identity] = edges
        self.emit_height[this_identity] = self.block_height
        

    # Bitcoin (esque) emission.
    def _get_emission(self, this_identity):
        
        # NOTE(const): Emit rate governs the average number of tokens to emit after each EOS block.
        # In the future, this amount should change with block height to model an emission decay
        # as is the case with Bitcoin.
        EMIT_RATE = 50              
        
        # The time since my last token emit. 
        # Note: if the emit rate changes in time then this calculation will need to be changed.
        delta_blocks = self.block_height - self.emit_height[this_identity]    
        
        # NOTE(const): The reinforcement term governs the effect of stake on the output. 
        # Removing this term makes all nodes equal. Dampening it with a logarithm will
        # decrease the effect of having a higher ratio of the total stake.
        stake_reinforcement = (self.stake[this_identity] / self.total_stake) 
        
        # NOTE(const): The emission term here is built to distribute rewards evenly across nodes
        # in proportion to their attribution in the graph (as well as a reflection of their stake holdings)
        this_emission = EMIT_RATE * delta_blocks * stake_reinforcement
        
        return this_emission
               
    def _do_emit(self, this_identity, this_emission):
        # NOTE(const): The _do_emit function is the most computationally expensive call within the contract. 
        # It involves a depth first recursion which terminates when the emission total has been exhausted. 
        # In the worst case this function's runtime is O(DELTA_BLOCKS * EMIT_RATE * (1 / MIN_INCREMENT))
        # The function is paid by the calling node which will require that node holds enough EOS to make the call.
        # TODO(const): I am uncertain of the EOS token requirements this function requires, or what happens when a 
        # a callee cannot afford the transaction.
        MIN_INCREMENT = 0.00001
              
        # Recurse the emmision through the tree.
        emission_queue = [(this_identity, this_emission)]
        while len(emission_queue) > 0:
            
            # Next.
            current_identity, current_emission = emission_queue.pop()
            current_inedge = self.edges[current_identity][0][1]
                        
            # NOTE(const): Update global vars for next step. In reality these will be updated immediately.
            # For testing purposes we hold them locked until every node has updated.
            self.next_stake[current_identity] += (current_emission * current_inedge) 
            self.next_total_stake += (current_emission * current_inedge)
             
            # Recurse.
            for next_identity, next_weight in self.edges[current_identity][1:]:
                next_emission = current_emission * next_weight                
                if next_emission > MIN_INCREMENT:
                    emission_queue.append((next_identity, current_emission * next_weight)) 
                                   
        
    def __str__(self):
        strng = "" 
        strng += "S: " + str_arr([self.stake[identity] for identity in self.stake.keys()]) + "\n" 
        return strng